# Hyperparameter Optimization nominal nov 23 div


In [2]:
import numpy as np
import torch
import optuna
import os
import pickle
import copy

import sys
sys.path.append('./data')
from datautil import sys_separation,create_cross_loaders

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Choose model

In [4]:
from architectures import RUL_Transformer as arch


from training import train_xu_y as train
from training import evaluate_xu_y as evaluate

# Choose data and experiment name

In [5]:
data_name='Rectifier_nov23_4steps_perform3'
sim_name='RectifierData_step10h' 
dataset_name='RUL_real'


exp='rul_real'#'try_verb' # 'try' #'end2end_optim'   #  # 
verbose=False #True#
exp_address=f'./Experiments/{data_name}/{sim_name}_{exp}'

## Load the dataset

In [6]:
with open(f'./data/{data_name}/dataset_{dataset_name}_train.pkl', 'rb') as file:
    train_dataset = pickle.load(file)

In [7]:
print(f'Dataset name: {train_dataset.dataset_name}') 
for var in train_dataset.n_var:
    if train_dataset.n_var[var]:
        print(f'Features ({var}): {train_dataset.labels[var]}')  
        print(f'-shape: {train_dataset.var[var].shape}')

if train_dataset.n_var['U']:
    simulation_size=train_dataset.origin['U'].shape[1]
    print(f'Simulation size: {simulation_size}') 
print(f'Number of systems: {train_dataset.n_sys}') 

Dataset name: RUL_real
Features (X): ['Ripple']
-shape: (62894, 1)
Features (U): ['clean R']
-shape: (62894, 400, 1)
Features (Y): ['RUL']
-shape: (62894, 1)
Simulation size: 400
Number of systems: 200


In [8]:
n_variables,n_latents,n_inputs,n_outputs=tuple(train_dataset.n_var.values())

#_,sim_life,n_inputs=dataset.u.shape
print(f'Number of variables: {n_variables}')
print(f'Number of latents: {n_latents}')
print(f'Number of inputs: {n_inputs}')
print(f'Number of outputs: {n_outputs}')

Number of variables: 1
Number of latents: 0
Number of inputs: 1
Number of outputs: 1


## Hyper-parameter Optimization

In [8]:
n_folds=5

#look_backward_list=[k for k in fdrange(30)]
#look_forward_list=[k for k in range(30)]
batch_size=64
stats=['X','U']
config_loaders={'n_folds':n_folds,'batch_size':batch_size,'norm_type':'minmax','stats':stats}
def objective(trial): 

    # model parameters   
    head_dim=trial.suggest_categorical('head_dim',[32, 64, 128, 256,512,1024]) #256 256 256 64
    n_head=trial.suggest_categorical('n_head',[1,2,4,6,8,9,10,11,12])                               #2   4  2 1
    param=trial.suggest_categorical('embed_dim/n_head',[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])        #7 2 5 5 
    embed_dim=param*n_head
    dim_feedforward=trial.suggest_categorical('dim_feedforward',[16,32, 64, 128, 256, 512,1024])   #32 #32 32 512
    p=trial.suggest_float('dropout',0,0.5) 

    mlp_dim1=trial.suggest_categorical('mlp_dim1',[32, 64, 128, 256,512,1024]) #256 256 256 64
    mlp_dim2=trial.suggest_categorical('mlp_dim2',[32, 64, 128, 256,512,1024]) #256 256 256 64
    u_enc_dim=trial.suggest_categorical('u_enc_dim',[1, 2, 4, 8,12,16])
    mlp_dim=[mlp_dim1,mlp_dim2]
    p_red=trial.suggest_float('p_red',0,0.5)


    #training parameters
    lr=trial.suggest_float('lr',1e-5,1e-3) 
    #Condition for tries
    if exp=='try':
        n_epoch=1
    else:
        n_epoch=trial.suggest_int('n_epoch',5,30) 


    # window
    look_backward=trial.suggest_int('look_backward',29,29) #
    #look_forward=0#trial.suggest_int('look_forward',0,0)   
    window={'X':[look_backward,0],'Y':[0,0]}

    u_input_dim=n_inputs*simulation_size
    x_input_dim=(look_backward+1)*n_variables
    output_dim=n_outputs
    

    # configuration
    config_model={'input_dim':u_input_dim,
                    'pred_window':output_dim,
                    'embed_dim':embed_dim,
                    'head_dim':head_dim,
                    'dim_feedforward':dim_feedforward,
                    'n_head':n_head,
                    'dropout':p,
                    'mlp_dim':mlp_dim,
                    'u_enc_dim':u_enc_dim,
                    'perform_w':x_input_dim,
                    'dropout_pred':p_red
                    }
    
    config_train={'n_epoch':n_epoch,'lr':lr,'window':window}
    config_loaders.update({'window':window})

    if verbose:
        print(f'Model: {config_model}')
        print(f'Training: {config_train}')


    #create loaders randomly 
    train_loaders,test_loaders=create_cross_loaders(train_dataset,**config_loaders,shuffle=True) #,train_stats   

    #cross validation
    y_preds=[]
    y_trues=[] ##### improve this
    weights=[]

    arrays=[]
    for fold in range(n_folds):
        #create the model


        #create the model
        model=arch(**config_model).to(device).double() 

        #train and evaluate model on fold 

        model_trained=train(model,train_loaders[fold],**config_train,verbose=verbose)
        weight=copy.deepcopy(model_trained.state_dict()) 

        y_pred,y_true=evaluate(model_trained,test_loaders[fold],window=window,verbose=verbose)
        
        #save folds predictions
        y_preds.append(y_pred)
        y_trues.append(y_true)
        weights.append(weight)
        arrays.append(test_loaders[fold].dataset.sys_array)

    y_pred=np.concatenate(y_preds,axis=0)
    y_true=np.concatenate(y_trues,axis=0)
    
    array=np.concatenate(arrays)
    
    #System separation
    y_pred_sep=sys_separation(y_pred,array)
    y_true_sep=sys_separation(y_true,array)

    #RMSE
    RMSE_sys=np.sqrt(np.nanmean((y_pred_sep-y_true_sep)**2,axis=tuple(range(1,y_pred_sep.ndim))))
    RMSE_mean=np.mean(RMSE_sys,axis=0)
    #RMSE_std=np.std(RMSE_sys,axis=0)

    #scoring
    # Nasa_scoring_sys=np.nanmean(scoring(y_pred,y_true),axis=1)
    # Nasa_scoring_mean=np.nanmean(Nasa_scoring_sys,axis=0)
    # Nasa_scoring_std=np.nanstd(Nasa_scoring_sys,axis=0)
    
    # score=0.5*RMSE_mean+0.5*Nasa_scoring_mean
    
    if trial.number==0 or study.best_trial.value >= RMSE_mean:
        torch.save(train_loaders, f'{exp_address}/train_loaders.pkl')
        torch.save(test_loaders, f'{exp_address}/test_loaders.pkl')
        torch.save(weights, f'{exp_address}/best_model1.pt')
        config={'loaders':config_loaders,'model':config_model,'train':config_train}
        torch.save(config, f'{exp_address}/best_config.pt')
    return RMSE_mean

In [9]:
study_name = exp_address+'/estudio' #f"{dataset_name}_test{test}"
storage_name = "sqlite:///{}.db".format(study_name)


os.makedirs(f'{exp_address}', exist_ok=True)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
len(study.trials)

[I 2023-11-19 19:49:02,629] Using an existing study with name './Experiments/Rectifier_nov23_4steps_perform3/RectifierData_step10h_rul_real/estudio' instead of creating a new one.


100

In [10]:
n_trials=100
study.optimize(objective, n_trials=n_trials-len(study.trials), show_progress_bar=True)

In [11]:
study.best_trial.params

{'head_dim': 32,
 'n_head': 6,
 'embed_dim/n_head': 12,
 'dim_feedforward': 128,
 'dropout': 0.47787675008320984,
 'mlp_dim1': 1024,
 'mlp_dim2': 1024,
 'u_enc_dim': 2,
 'p_red': 0.2256869271809978,
 'lr': 0.0009524868575007632,
 'n_epoch': 28,
 'look_backward': 29}

In [16]:
study.best_trial.value

6.483713908864895